# DATA PROCESSING

El objetivo de este notebook es cargar cada uno de los archivos csv que contienen los datos de los diferentes datasets y realizar un preprocesamiento de los mismos para posteriormente combinarlos en un único dataFrame.

En el preprocesamiento se realizarán las siguientes tareas:
- Cargar los datos en un dataFrame
- Eliminar las columnas o filas de datos totales
- Eliminar los datos a nivel nacional
- Eliminar las columnas con el nombre de la comunidad autónoma para evitar duplicidad.

## Preparación del entorno

In [246]:
import pandas as pd
import os 

RAW_DATA_PATH = os.path.join("..", "data/raw/")
SILVER_DATA_PATH = os.path.join("..", "data/silver/")

In [247]:
comunidades_eliminar = ["Ceuta", "Melilla", "Canarias", "Balears, Illes"]
provincias_eliminar = ["Ceuta", "Melilla", "Palmas, Las", "Santa Cruz de Tenerife", "Balears, Illes"]

## Consumo Electrico Anual

In [248]:
import pandas as pd

consumo_df = pd.read_csv(RAW_DATA_PATH+ "ConsumoElectricoAnual.csv", sep="\t", encoding="latin1")
consumo_df.rename(columns={"Comunidades y Ciudades Autï¿½nomas": "Comunidades Autónomas", "aï¿½o": "Año"}, inplace=True) #linux
consumo_df.rename(columns={"Comunidades y Ciudades Autónomas": "Comunidades Autónomas", "año": "Año"}, inplace=True) #windows
#nota: por alguna razón, el encoding funciona distinto, la función 'rename' ignora los errores si la columna no tiene el mismo nombre
consumo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Total Nacional         100 non-null    object
 1   Comunidades Autónomas  95 non-null     object
 2   Producto consumido     100 non-null    object
 3   Año                    100 non-null    int64 
 4   Total                  100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [249]:
consumo_df.head(7)

,Total Nacional,Comunidades Autónomas,Producto consumido,Año,Total
0,Total Nacional,NaN,Electricidad,2022,14.319.055
1,Total Nacional,NaN,Electricidad,2021,8.892.228
2,Total Nacional,NaN,Electricidad,2019,6.367.976
3,Total Nacional,NaN,Electricidad,2017,6.227.459
4,Total Nacional,NaN,Electricidad,2015,6.177.084
5,Total Nacional,01 Andalucía,Electricidad,2022,1.653.989
6,Total Nacional,01 Andalucía,Electricidad,2021,941.599


Comprobación de si hay datos nulos

In [250]:
print(consumo_df.isnull().sum())

Total Nacional           0
Comunidades Autónomas    5
Producto consumido       0
Año                      0
Total                    0
dtype: int64


Eliminamos los valores nulos en las comunidades autónomas, que representan el total nacional.

In [251]:
consumo_df.dropna(subset=["Comunidades Autónomas"], inplace=True)

Eliminamos las columnas "Total Nacional" y "Producto consumido", ya que, todos los registros contienen el mismo valor en ambas columnas

In [252]:
consumo_df.drop(columns=["Total Nacional", "Producto consumido"], inplace=True)

Para cada valor de la columna "Comunidades y Ciudades Autónomas", eliminamos los 3 primeros carácteres

In [253]:
consumo_df["Comunidades Autónomas"] = consumo_df["Comunidades Autónomas"].str.replace(r"\b\d{2} ", "", regex=True)

In [254]:
consumo_df.head()

,Comunidades Autónomas,Año,Total
5,Andalucía,2022,1.653.989
6,Andalucía,2021,941.599
7,Andalucía,2019,637.486
8,Andalucía,2017,603.104
9,Andalucía,2015,572.742


Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las comunidades que no están en la península.

In [255]:
consumo_df.set_index("Comunidades Autónomas", inplace=True)
consumo_df = consumo_df[~consumo_df.index.isin(comunidades_eliminar)]

In [256]:
with open(SILVER_DATA_PATH + "consumo_electrico.csv", "w") as f:
    consumo_df.to_csv(f, sep=';', encoding='utf-8')

## Dispositivos de energía renovable

In [257]:
dispositivos_df = pd.read_csv(RAW_DATA_PATH + "Dispositivos de energía renovable.csv", sep=";", encoding="latin")
dispositivos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4770 entries, 0 to 4769
Data columns (total 5 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Provincias                                   4770 non-null   object
 1   Nivel de ingresos mensuales netos del hogar  4770 non-null   object
 2   Tamaño del municipio                         4770 non-null   object
 3   Tipo de dispositivo de energía renovable     4770 non-null   object
 4   Total                                        4770 non-null   object
dtypes: object(5)
memory usage: 186.5+ KB


In [258]:
dispositivos_df.head()

,Provincias,Nivel de ingresos mensuales netos del hogar,Tamaño del municipio,Tipo de dispositivo de energía renovable,Total
0,Total Nacional,Total,Total,Un único tipo de dispositivo: Energía solar té...,736.984
1,Total Nacional,Total,Total,Un único tipo de dispositivo: Energía solar fo...,323.853
2,Total Nacional,Total,Total,Combinación de varios tipos de dispositivos,44.205
3,Total Nacional,Total,Total,No dispone de dispositivo de energía renovable,17.705.004
4,Total Nacional,Total,Total,Total,18.810.046


Comprobamos que no haya valores nules ni columnas que no nos aporten información relevante


In [259]:
print(dispositivos_df.isnull().sum())

Provincias                                     0
Nivel de ingresos mensuales netos del hogar    0
Tamaño del municipio                           0
Tipo de dispositivo de energía renovable       0
Total                                          0
dtype: int64


In [260]:
print(dispositivos_df.nunique())

Provincias                                       53
Nivel de ingresos mensuales netos del hogar       6
Tamaño del municipio                              3
Tipo de dispositivo de energía renovable          5
Total                                          3136
dtype: int64


Eliminamos lso valores de Provincias que sean el Total Nacional ya que no vamos a necesitarlo

In [261]:
dispositivos_df.drop(dispositivos_df[dispositivos_df["Provincias"] == "Total Nacional"].index, inplace=True)
dispositivos_df.head()

,Provincias,Nivel de ingresos mensuales netos del hogar,Tamaño del municipio,Tipo de dispositivo de energía renovable,Total
90,Albacete,Total,Total,Un único tipo de dispositivo: Energía solar té...,5.061
91,Albacete,Total,Total,Un único tipo de dispositivo: Energía solar fo...,1.193
92,Albacete,Total,Total,Combinación de varios tipos de dispositivos,342
93,Albacete,Total,Total,No dispone de dispositivo de energía renovable,146.254
94,Albacete,Total,Total,Total,152.850


## Distribución de edad de la población española

In [262]:
edad_df=pd.read_csv(RAW_DATA_PATH+"Distribución de edad de la población española.csv", sep = ";", encoding = "latin")

edad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Total Nacional                    12672 non-null  object
 1   Comunidades y Ciudades Autónomas  12496 non-null  object
 2   Provincias                        9152 non-null   object
 3   Españoles/Extranjeros             12672 non-null  object
 4   Edad (grupos quinquenales)        12672 non-null  object
 5   Sexo                              12672 non-null  object
 6   Periodo                           12672 non-null  object
 7   Total                             12672 non-null  object
dtypes: object(8)
memory usage: 792.1+ KB


In [263]:
# ejemplo de los datos de la tabla para una provincia
edad_df.head(600).tail(8)

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Españoles/Extranjeros,Edad (grupos quinquenales),Sexo,Periodo,Total
592,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2022,83.985
593,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2021,87.048
594,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2020,90.338
595,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2019,93.464
596,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2018,96.477
597,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2017,99.754
598,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2016,102.768
599,Total Nacional,01 Andalucía,11 Cádiz,Total,De 35 a 39 años,Total,1 de enero de 2015,104.523


In [264]:
print(edad_df[['Total Nacional', 'Españoles/Extranjeros','Sexo']].nunique())

Total Nacional           1
Españoles/Extranjeros    1
Sexo                     1
dtype: int64


Observamos que las columnas 'Total Nacional', 'Españoles/Extranjeros' y 'Sexo' no aportan ninguna información, por lo que se pueden eliminar.

In [265]:
edad_df.drop(columns=['Total Nacional', 'Españoles/Extranjeros', 'Sexo'], inplace=True)

Hemos decidido quedarnos solo con los datos del 2022, por lo que se pueden eliminar las filas de otros años y después la columna 'Periodo'.

In [266]:
edad_df.drop(edad_df[edad_df['Periodo'] != '1 de enero de 2022'].index, inplace=True)
edad_df.drop(columns=['Periodo'], inplace=True)

Los primeros registros del dataset son los datos a nivel nacional, donde la comunidad autónoma y la provincia son valores nulos, por lo que se pueden eliminar. Además, se eliminará la columna 'Comunidades y Ciudades Autónomas' y las filas con los totales en 'Edad'.

In [267]:
edad_df.head()

,Comunidades y Ciudades Autónomas,Provincias,Edad (grupos quinquenales),Total
0,NaN,NaN,Todas las edades,47.475.420
8,NaN,NaN,De 0 a 4 años,1.809.768
16,NaN,NaN,De 5 a 9 años,2.236.911
24,NaN,NaN,De 10 a 14 años,2.517.373
32,NaN,NaN,De 15 a 19 años,2.471.599


In [268]:
edad_df.dropna(subset=["Comunidades y Ciudades Autónomas", "Provincias"], inplace=True)
edad_df.drop(columns=['Comunidades y Ciudades Autónomas'], inplace=True)
edad_df.drop(edad_df[edad_df['Edad (grupos quinquenales)'] == 'Todas las edades'].index, inplace=True)

Se eliminará el número que identifica a cada provincia para dejar solo el nombre.

In [269]:
edad_df["Provincias"] = edad_df["Provincias"].str.replace(r"^\d+\s", "", regex=True)

Una vez finalizado el preprocesamiento, el dataset queda con 1092 entradas y 3 columnas.

In [270]:
edad_df.head()

,Provincias,Edad (grupos quinquenales),Total
360,Almería,De 0 a 4 años,36.360
368,Almería,De 5 a 9 años,41.033
376,Almería,De 10 a 14 años,43.411
384,Almería,De 15 a 19 años,40.722
392,Almería,De 20 a 24 años,41.532


In [271]:
edad_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1092 entries, 360 to 12664
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Provincias                  1092 non-null   object
 1   Edad (grupos quinquenales)  1092 non-null   object
 2   Total                       1092 non-null   object
dtypes: object(3)
memory usage: 34.1+ KB


Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [272]:
edad_df.set_index("Provincias", inplace=True)
edad_df = edad_df[~edad_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "distribucion_edad.csv", "w") as f:
    edad_df.to_csv(f, sep=';', encoding='utf-8')

## Intensidad de uso en viviendas

In [274]:
import pandas as pd 

intensidad_df = pd.read_csv(RAW_DATA_PATH + "Intensidad de uso en viviendas.csv", sep=";", encoding="latin1")
intensidad_df.rename(columns={"Comunidades y Ciudades Autónomas": "Comunidades Autónomas"}, inplace=True)
intensidad_df.head()

,Total Nacional,Comunidades Autónomas,Provincias,Municipios,Consumo eléctrico,Total
0,Total Nacional,NaN,NaN,NaN,Viviendas totales,26.623.708
1,Total Nacional,NaN,NaN,NaN,Viviendas vacías,3.837.328
2,Total Nacional,NaN,NaN,NaN,Mediana consumo anual,2.196
3,Total Nacional,NaN,NaN,NaN,Viviendas con bajo consumo,943.924
4,Total Nacional,NaN,NaN,NaN,Viviendas de uso esporádico,2.514.511


Comprobamos si hay valores nulos en cada una de las columnas del dataframe

In [275]:
intensidad_df.isnull().sum()

Total Nacional             0
Comunidades Autónomas     18
Provincias                18
Municipios               954
Consumo eléctrico          0
Total                      0
dtype: int64

In [276]:
len(intensidad_df["Comunidades Autónomas"].tolist())

954

En este proyecto se realizarán una serie de analísis de datos teniendo en cuenta únicamente las comunidades autónomas y las provincias, por lo que se elimina la columna "Municipios",ya que no nos ofrece información útil para el estudio.

Además, se elimina la columna "Total Nacional" ya que tampoco aporta ninguna información.

In [277]:
intensidad_df.drop(columns=["Municipios","Total Nacional"], inplace=True)

Eliminamos las filas con valores en "Provincia" y "Comunidad Autónoma" nulos. Además de la columna Comunidad Autónoma.

In [278]:
intensidad_df.dropna(subset=["Provincias"],inplace=True)
intensidad_df.drop(columns=["Comunidades Autónomas"], inplace=True)

Quitamos el número de la provincia para dejar solo el nombre.

In [279]:
intensidad_df["Provincias"] = intensidad_df["Provincias"].str.replace(r"^\d+\s", "", regex=True)

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [280]:
intensidad_df.set_index("Provincias", inplace=True)
intensidad_df = intensidad_df[~intensidad_df.index.isin(provincias_eliminar)]

In [281]:
with open(SILVER_DATA_PATH + "intensidad_de_uso_en_viviendas.csv", "w") as f:
    intensidad_df.to_csv(f, sep=';', encoding='utf-8')

## Producción de energía por lugar

In [282]:
produccion_df = pd.read_csv(RAW_DATA_PATH + "Producción de energía por lugar.csv", sep=",", encoding="utf-8")
produccion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10595 entries, 0 to 10594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Latitud             10595 non-null  float64
 1   Longitud            10595 non-null  float64
 2   Potencia            10594 non-null  object 
 3   Comunidad Autonoma  10595 non-null  object 
 4   Provincia           10595 non-null  object 
dtypes: float64(2), object(3)
memory usage: 414.0+ KB


In [283]:
produccion_df.head()

,Latitud,Longitud,Potencia,Comunidad Autonoma,Provincia
0,36.1,-6.05,PV outputJanuary: 116.96 kWh,Comunidad autónoma no encontrada,Provincia no encontrada
1,36.1,-6.05,PV outputFebruary: 120.11 kWh,Comunidad autónoma no encontrada,Provincia no encontrada
2,36.1,-6.05,PV outputMarch: 143.07 kWh,Comunidad autónoma no encontrada,Provincia no encontrada
3,36.1,-6.05,PV outputApril: 154.15 kWh,Comunidad autónoma no encontrada,Provincia no encontrada
4,36.1,-6.05,PV outputMay: 165.48 kWh,Comunidad autónoma no encontrada,Provincia no encontrada


Comporobamos si hay valores nulos o columnas que no nos aporten información

In [284]:
print(produccion_df.isnull().sum())

Latitud               0
Longitud              0
Potencia              1
Comunidad Autonoma    0
Provincia             0
dtype: int64


In [285]:
print(produccion_df.nunique())

Latitud                 29
Longitud                41
Potencia              9004
Comunidad Autonoma      28
Provincia               44
dtype: int64


Eliminamos los valores nulos de Potencia y de Provincia y Comunidad Autonoma los no encontrados

Eliminamos también las columnas Latitud y Longitud

In [286]:
produccion_df.dropna(subset=["Potencia"], inplace=True)
produccion_df.drop(produccion_df[produccion_df["Provincia"] == "Provincia no encontrada"].index, inplace=True)
produccion_df.drop(produccion_df[produccion_df["Comunidad Autonoma"] == "Comunidad autónoma no encontrada"].index, inplace=True)
produccion_df.drop(columns=["Latitud", "Longitud"], inplace=True)

Separamos los valores de Potencia en Mes y Potencia kWh 

In [287]:
produccion_df["Mes"] = produccion_df["Potencia"].str.extract(r'^PV output([^:]+)')
produccion_df["Potencia"] = produccion_df["Potencia"].str.extract(r":\s*([\d.]+)").astype(float)
produccion_df.rename(columns={"Potencia": "Potencia kWh"}, inplace=True)
produccion_df.head(24)

,Potencia kWh,Comunidad Autonoma,Provincia,Mes
12,104.97,Andalucía,Cádiz,January
13,113.40,Andalucía,Cádiz,February
14,137.00,Andalucía,Cádiz,March
15,147.93,Andalucía,Cádiz,April
16,161.61,Andalucía,Cádiz,May
17,158.61,Andalucía,Cádiz,June
18,165.98,Andalucía,Cádiz,July
19,163.39,Andalucía,Cádiz,August
20,147.51,Andalucía,Cádiz,September
21,132.53,Andalucía,Cádiz,October


Como podemos observar que hay varios valores para la misma provincia, esto se debe a que el dataset lo divide por latitudes y longitudes. Hacemos un sumatorio para tener los valores totales por provincia 

In [288]:
produccion_df["Potencia kWh"] = produccion_df.groupby(["Comunidad Autonoma", "Provincia", "Mes"])["Potencia kWh"].transform("mean")
produccion_df.drop_duplicates(subset=["Comunidad Autonoma", "Provincia", "Mes"], inplace=True)
produccion_df.head(24)

,Potencia kWh,Comunidad Autonoma,Provincia,Mes
12,112.198462,Andalucía,Cádiz,January
13,112.970769,Andalucía,Cádiz,February
14,137.280769,Andalucía,Cádiz,March
15,143.985385,Andalucía,Cádiz,April
16,158.723846,Andalucía,Cádiz,May
17,157.848462,Andalucía,Cádiz,June
18,168.123846,Andalucía,Cádiz,July
19,165.183846,Andalucía,Cádiz,August
20,146.355385,Andalucía,Cádiz,September
21,131.309231,Andalucía,Cádiz,October


In [291]:
produccion_df.drop(columns = ["Comunidad Autonoma"], inplace=True)

## Renta por hogar 

In [170]:
renta_df=pd.read_csv(RAW_DATA_PATH+"Renta media por hogar.csv", sep = ";", encoding = "latin")

renta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Comunidades y Ciudades Autónomas  186 non-null    object 
 1   Provincias                        186 non-null    object 
 2   Islas                             30 non-null     object 
 3   Indicadores de renta media        186 non-null    object 
 4   Periodo                           186 non-null    int64  
 5   Total                             185 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 8.8+ KB


In [171]:
# Ejemplo de los datos de la tabla para una provincia
renta_df.head(68).tail(8)

,Comunidades y Ciudades Autónomas,Provincias,Islas,Indicadores de renta media,Periodo,Total
60,Canarias,Santa Cruz de Tenerife,NaN,Renta neta media por hogar,2022,32.485
61,Canarias,Santa Cruz de Tenerife,NaN,Renta neta media por hogar,2021,29.849
62,Canarias,Santa Cruz de Tenerife,NaN,Renta neta media por hogar,2020,28.792
63,Canarias,Santa Cruz de Tenerife,"Gomera, La",Renta neta media por hogar,2022,30.156
64,Canarias,Santa Cruz de Tenerife,"Gomera, La",Renta neta media por hogar,2021,28.054
65,Canarias,Santa Cruz de Tenerife,"Gomera, La",Renta neta media por hogar,2020,26.509
66,Canarias,Santa Cruz de Tenerife,"Hierro, El",Renta neta media por hogar,2022,31.010
67,Canarias,Santa Cruz de Tenerife,"Hierro, El",Renta neta media por hogar,2021,29.903


In [172]:
#Eliminamos columna Islas, pues no es de interés.
renta_df.drop(columns=['Islas'], inplace=True)

#Eliminar la columna "Renta anual media por hogar" pues no es relevante.
renta_df.drop(columns='Indicadores de renta media', inplace=True)

#Eliminar la columna "Comunidades y Ciudades Autónomas".
renta_df.drop(columns='Comunidades y Ciudades Autónomas', inplace=True)

renta_df.head()

,Provincias,Periodo,Total
0,Almería,2022,28.442
1,Almería,2021,27.184
2,Almería,2020,26.276
3,Cádiz,2022,30.055
4,Cádiz,2021,28.501


In [173]:
#Como la provincia de Navarra en el año 2020 tiene un valor nulo, se añade su correspondiente valor, consultado en Nastat.
#Ver valor primero

# Localizamos la fila específica para Navarra en 2020 y le asignamos un nuevo valor en la columna "Total"
renta_df.loc[(renta_df['Provincias'] == 'Navarra') & (renta_df['Periodo'] == 2020), 'Total'] = 37082

renta_df.loc[(renta_df['Provincias'] == 'Navarra') & (renta_df['Periodo'] == 2020), 'Total']

167    37082.0
Name: Total, dtype: float64

In [174]:
#Información actual de nuestro dataframe:
renta_df.info()

#Cómo se ve el dataframe ahora:
renta_df.head(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Provincias  186 non-null    object 
 1   Periodo     186 non-null    int64  
 2   Total       186 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.5+ KB


,Provincias,Periodo,Total
0,Almería,2022,28.442
1,Almería,2021,27.184
2,Almería,2020,26.276
3,Cádiz,2022,30.055
4,Cádiz,2021,28.501
5,Cádiz,2020,27.179
6,Córdoba,2022,29.082
7,Córdoba,2021,27.909


Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [296]:
renta_df.set_index("Provincias", inplace=True)
renta_df = renta_df[~renta_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "renta_media_hogar.csv", "w") as f:
    renta_df.to_csv(f, sep=';', encoding='utf-8')

## Tipo de núcleo familiar

In [175]:
tipo_nucleo_familiar_df=pd.read_csv(RAW_DATA_PATH+"Tipo de núcleo familiar.csv", sep = ";", encoding = "latin")

tipo_nucleo_familiar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855 entries, 0 to 1854
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Provincias               1855 non-null   object
 1   Tipo de núcleo familiar  1855 non-null   object
 2   Número de hijos          1855 non-null   object
 3   Total                    1855 non-null   object
dtypes: object(4)
memory usage: 58.1+ KB


In [176]:
# Ejemplo de los datos que presenta la tabla
tipo_nucleo_familiar_df.head(800).tail(10)

,Provincias,Tipo de núcleo familiar,Número de hijos,Total
790,Granada,"Pareja no casada, con o sin hijos convivientes",Total,35.412
791,Granada,Padre sólo con hijos convivientes,0 hijos conviviendo,0
792,Granada,Padre sólo con hijos convivientes,1 hijo conviviendo,4.959
793,Granada,Padre sólo con hijos convivientes,2 o más hijos conviviendo,3.217
794,Granada,Padre sólo con hijos convivientes,0 hijos conviviendo menores de 25 años,4.200
795,Granada,Padre sólo con hijos convivientes,1 hijo conviviendo menor de 25 años,2.442
796,Granada,Padre sólo con hijos convivientes,2 hijos o más conviviendo menores de 25 años,1.534
797,Granada,Padre sólo con hijos convivientes,Total,8.176
798,Granada,Madre sóla con hijos convivientes,0 hijos conviviendo,0
799,Granada,Madre sóla con hijos convivientes,1 hijo conviviendo,28.188


En la columna "Provincias", algunas filas están etiquetadas como "Total Nacional". Para simplificar el conjunto de datos, se ha decidido eliminar los registros correspondientes a los datos a nivel nacional.

In [177]:
tipo_nucleo_familiar_df.head(5)

,Provincias,Tipo de núcleo familiar,Número de hijos,Total
0,Total Nacional,Total,0 hijos conviviendo,4.484.568
1,Total Nacional,Total,1 hijo conviviendo,4.914.044
2,Total Nacional,Total,2 o más hijos conviviendo,4.368.298
3,Total Nacional,Total,0 hijos conviviendo menores de 25 años,6.973.068
4,Total Nacional,Total,1 hijo conviviendo menor de 25 años,3.349.425


In [178]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Provincias'] != "Total Nacional"]


In [179]:
tipo_nucleo_familiar_df.head(10)

,Provincias,Tipo de núcleo familiar,Número de hijos,Total
35,Albacete,Total,0 hijos conviviendo,33.012
36,Albacete,Total,1 hijo conviviendo,35.351
37,Albacete,Total,2 o más hijos conviviendo,41.228
38,Albacete,Total,0 hijos conviviendo menores de 25 años,52.429
39,Albacete,Total,1 hijo conviviendo menor de 25 años,25.451
40,Albacete,Total,2 hijos o más conviviendo menores de 25 años,31.712
41,Albacete,Total,Total,109.592
42,Albacete,"Pareja casada, con o sin hijos convivientes",0 hijos conviviendo,27.344
43,Albacete,"Pareja casada, con o sin hijos convivientes",1 hijo conviviendo,21.719
44,Albacete,"Pareja casada, con o sin hijos convivientes",2 o más hijos conviviendo,31.743


En la columna "Tipo de núcleo familiar" se ha encontrado un valor "Total", el cual será eliminado.

In [180]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Tipo de núcleo familiar'] != "Total"]


Lo mismo sucede con la columna "Número de hijos".

In [181]:
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Número de hijos'] != "Total"]

En el dataset, el símbolo '.' debe interpretarse como dato que no se proporciona por muestra insuficiente.

In [182]:
columnas_a_verificar = ['Provincias', 'Tipo de núcleo familiar', 'Número de hijos', 'Total']
tipo_nucleo_familiar_df[columnas_a_verificar].apply(lambda x: x == '.').any()

Provincias                 False
Tipo de núcleo familiar    False
Número de hijos            False
Total                      False
dtype: bool

No se han encontrado valores nulos.

Una vez finalizado el preprocesamiento, el dataset queda con 1248 entradas y 4 columnas.

In [183]:
tipo_nucleo_familiar_df.head(10)


,Provincias,Tipo de núcleo familiar,Número de hijos,Total
42,Albacete,"Pareja casada, con o sin hijos convivientes",0 hijos conviviendo,27.344
43,Albacete,"Pareja casada, con o sin hijos convivientes",1 hijo conviviendo,21.719
44,Albacete,"Pareja casada, con o sin hijos convivientes",2 o más hijos conviviendo,31.743
45,Albacete,"Pareja casada, con o sin hijos convivientes",0 hijos conviviendo menores de 25 años,39.498
46,Albacete,"Pareja casada, con o sin hijos convivientes",1 hijo conviviendo menor de 25 años,16.930
47,Albacete,"Pareja casada, con o sin hijos convivientes",2 hijos o más conviviendo menores de 25 años,24.377
49,Albacete,"Pareja no casada, con o sin hijos convivientes",0 hijos conviviendo,5.669
50,Albacete,"Pareja no casada, con o sin hijos convivientes",1 hijo conviviendo,3.085
51,Albacete,"Pareja no casada, con o sin hijos convivientes",2 o más hijos conviviendo,3.555
52,Albacete,"Pareja no casada, con o sin hijos convivientes",0 hijos conviviendo menores de 25 años,5.669


In [184]:
a = tipo_nucleo_familiar_df['Tipo de núcleo familiar'].unique()
print(a)    

['Pareja casada, con o sin hijos convivientes'
 'Pareja no casada, con o sin hijos convivientes'
 'Padre sólo con hijos convivientes' 'Madre sóla con hijos convivientes']


Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
tipo_nucleo_familiar_df.set_index("Provincias", inplace=True)
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[~tipo_nucleo_familiar_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "tipo_nucleo_familiar.csv", "w") as f:
    tipo_nucleo_familiar_df.to_csv(f, sep=';', encoding='utf-8')

Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
intensidad_df.set_index("Provincias", inplace=True)
intensidad_df = intensidad_df[~intensidad_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "intensidad_de_uso_en_viviendas.csv", "w") as f:
    intensidad_df.to_csv(f, sep=';', encoding='utf-8')

## Viviendas según el número de personas

In [185]:
viviendas_numero_personas_df=pd.read_csv(RAW_DATA_PATH+"Viviendas según el número de personas.csv", sep = ";", encoding = "latin")

viviendas_numero_personas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Total Nacional                    1764 non-null   object
 1   Comunidades y Ciudades Autónomas  1736 non-null   object
 2   Provincias                        1204 non-null   object
 3   Tamaño del hogar                  1764 non-null   object
 4   Año                               1764 non-null   int64 
 5   Total                             1764 non-null   object
dtypes: int64(1), object(5)
memory usage: 82.8+ KB


In [186]:
# Ejemplo de los datos que presenta la tabla
viviendas_numero_personas_df.head(10)

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Tamaño del hogar,Año,Total
0,Total Nacional,NaN,NaN,Total,2023,18.849.621
1,Total Nacional,NaN,NaN,Total,2022,18.791.675
2,Total Nacional,NaN,NaN,Total,2021,18.736.673
3,Total Nacional,NaN,NaN,Total,2020,18.692.279
4,Total Nacional,NaN,NaN,1 persona,2023,4.987.240
5,Total Nacional,NaN,NaN,1 persona,2022,4.933.214
6,Total Nacional,NaN,NaN,1 persona,2021,4.881.986
7,Total Nacional,NaN,NaN,1 persona,2020,4.829.600
8,Total Nacional,NaN,NaN,2 personas,2023,5.804.559
9,Total Nacional,NaN,NaN,2 personas,2022,5.763.142


In [187]:
print(viviendas_numero_personas_df[['Total Nacional']].nunique())

Total Nacional    1
dtype: int64


La columna "Total Nacional" se eliminará ya que no añade valor al conjunto de datos.

In [188]:
viviendas_numero_personas_df.drop(columns=['Total Nacional'], inplace=True)

Se van a eliminar las primeras 28 filas del dataset ya que corresponden a los datos totales nacionales.

In [189]:
viviendas_numero_personas_df = viviendas_numero_personas_df.drop(index=range(0, 28))

Para cada comunidad autónoma, hay filas que corresponden al total de dicha comunidad, lo cual se representa con valores nulos en la columna "Provincias". El inconveniente es que, en comunidades con solo una provincia, los valores de la columna "Provincias" también son nulos. Por esta razón, se desea asignar el nombre de la provincia a las filas de comunidades autónomas que cuentan con una sola provincia y eliminar los valores totales en las comunidades autónomas con varias provincias.

In [190]:
# Definir el diccionario de asignaciones
asignaciones = {
    "03 Asturias, Principado de": "Asturias",
    "04 Balears, Illes": "Balears, Illes",
    "06 Cantabria": "Cantabria",
    "13 Madrid, Comunidad de": "Murcia",
    "14 Murcia, Región de": "Madrid",
    "15 Navarra, Comunidad Foral de":"Navarra", 
    "17 Rioja, La": "Rioja, La",
    "18 Ceuta": "Ceuta",
    "19 Melilla": "Melilla"
}

# Identificar las comunidades autónomas con una sola provincia
comunidades_uniprovinciales = set(asignaciones.keys())

# Reemplazar valores nulos en "Provincias" para comunidades uniprovinciales
for comunidad, provincia in asignaciones.items():
    viviendas_numero_personas_df.loc[(viviendas_numero_personas_df["Comunidades y Ciudades Autónomas"] == comunidad) & (viviendas_numero_personas_df["Provincias"].isnull()), "Provincias"] = provincia

# Eliminar filas de valores totales por comunidad autónoma
viviendas_numero_personas_df = viviendas_numero_personas_df.dropna(subset=["Provincias"])


Se elimina la columna por comunidad autónoma.

In [191]:
viviendas_numero_personas_df = viviendas_numero_personas_df.drop(columns=["Comunidades y Ciudades Autónomas"])

En la columna "Tamaño del hogar", hay filas que contienen el total y otras que indican el tamaño medio. Se eliminarán aquellas que cumplan con cualquiera de estas condiciones.

In [192]:
viviendas_numero_personas_df = viviendas_numero_personas_df[viviendas_numero_personas_df['Tamaño del hogar'] != "Total"]
viviendas_numero_personas_df = viviendas_numero_personas_df[viviendas_numero_personas_df['Tamaño del hogar'] != "Tamaño medio"]

Hemos decidido quedarnos solo con los datos del 2022, por lo que se pueden eliminar las filas de otros años y después la columna 'Año'.

In [193]:
viviendas_numero_personas_df.drop(viviendas_numero_personas_df[viviendas_numero_personas_df['Año'] != 2022].index, inplace=True)
viviendas_numero_personas_df.drop(columns=['Año'], inplace=True)

Una vez finalizado el preprocesamiento, el dataset queda con 260 entradas y 3 columnas.

In [194]:
viviendas_numero_personas_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260 entries, 61 to 1757
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Provincias        260 non-null    object
 1   Tamaño del hogar  260 non-null    object
 2   Total             260 non-null    object
dtypes: object(3)
memory usage: 8.1+ KB


In [195]:
viviendas_numero_personas_df.head(10)

,Provincias,Tamaño del hogar,Total
61,04 Almería,1 persona,62.352
65,04 Almería,2 personas,77.714
69,04 Almería,3 personas,54.438
73,04 Almería,4 personas,51.754
77,04 Almería,5 personas o más,22.374
89,11 Cádiz,1 persona,109.454
93,11 Cádiz,2 personas,131.124
97,11 Cádiz,3 personas,108.196
101,11 Cádiz,4 personas,98.965
105,11 Cádiz,5 personas o más,28.595


Transformamos el dataframe para que cada fila sea una comunidad autónoma y cada columna un año, en cada celda estará el valor total. También eliminamos las provincias que no están en la península.

In [ ]:
viviendas_numero_personas_df.set_index("Provincias", inplace=True)
viviendas_numero_personas_df = viviendas_numero_personas_df[~viviendas_numero_personas_df.index.isin(provincias_eliminar)]

In [ ]:
with open(SILVER_DATA_PATH + "viviendas_personas.csv", "w") as f:
    viviendas_numero_personas_df.to_csv(f, sep=';', encoding='utf-8')